In [ ]:
import numpy as np
import airo_models
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker

In [ ]:
def AddPlanarBinAndSimpleBox(plant, mass=0.411, mu=1.0, width=0.2, depth=0.05, height=0.3):
    parser = Parser(plant)
    bin = parser.AddModelFromFile(FindResource("models/planar_bin.sdf"))
    plant.WeldFrames(plant.world_frame(),
                     plant.GetFrameByName("bin_base", bin),
                     RigidTransform(RotationMatrix.MakeZRotation(np.pi / 2.0),
                                    [0, 0, -0.035]))

    planar_joint_frame = plant.AddFrame(
        FixedOffsetFrame(
            "planar_joint_frame", plant.world_frame(),
            RigidTransform(RotationMatrix.MakeXRotation(np.pi / 2))))
    box_instance = AddShape(plant, Box(width, depth, height), "box", mass, mu)
    box_joint = plant.AddJoint(PlanarJoint(
        "box_joint", planar_joint_frame,
        plant.GetFrameByName("box", box_instance)))
    box_joint.set_default_translation([0, height/2.0])
    return box_instance